<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/Classificazione_in_localizzabile_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usiamo ResNet50 per classificare le immagini in localizzabili oppure no

In [1]:
import os
import sys
from pathlib import Path

# with this line we can check if we are in colab or not
in_colab = 'google.colab' in sys.modules
print("are we in Colab?:",in_colab)

cwd = Path(os.getcwd())
if in_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    os.chdir('/gdrive/MyDrive/GeoEstimation')
    
else:
    #our defult wd in local should be Vision_Project
    if str(cwd)[-14:] == 'Vision_Project':
        os.chdir(cwd / 'GeoEstimation')
    sys.path.append(cwd / 'GeoEstimation')

are we in Colab?: True
Mounted at /gdrive
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import sys
from pathlib import Path
from PIL import Image
from numpy import asarray
import multiprocessing as mp

In [2]:
dataset = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/im2gps3k_places365_classified.csv").dropna()

In [8]:
dataset2 = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/yfcc4k.csv").dropna()

In [4]:
images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]

In [9]:
images2 = ["/gdrive/MyDrive/GeoEstimation/resources/images/yfcc4k/{}".format(img) for img in dataset['IMG_ID']]

In [ ]:
#images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]
#img_all = [Image.open(image) for image in images]
#img_data = [asarray(img) for img in img_all]

In [ ]:
# a function to load all the images in an array format in img_array
# other "easier solutions" were really slow, this works fine
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.resize((256, 256))
        return np.asarray(img)

# Crea una pool di processi
with mp.Pool() as pool:
  # Carica le immagini in parallelo
  images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]
  img_data = pool.map(load_image, images)

# Crea un array multidimensionale a partire dalla lista di array numpy
img_array = np.stack(img_data)


In [6]:
# now we can re-set the GPU
import tensorflow as tf
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [7]:
img_array.shape

(2997, 256, 256, 3)

In [8]:
# the labels
label_array = np.array(dataset['LOC']).reshape((2997, 1))

In [9]:
labels_int = label_array.astype(int)

# Conto gli zeri, gli uni e i due nell'array
count = np.bincount(labels_int.flatten())

# Stampo i risultati
print(f"Zeri: {count[0]}")
print(f"Uni: {count[1]}")
print(f"Due: {count[2]}")

Zeri: 805
Uni: 671
Due: 1521


In [10]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(img_array, label_array, test_size=0.16, random_state=123)

In [11]:
y_int = y_train.astype(int)

# Conto gli zeri, gli uni e i due nell'array
count = np.bincount(y_int.flatten())

# Stampo i risultati
print(f"Zeri: {count[0]}")
print(f"Uni: {count[1]}")
print(f"Due: {count[2]}")

Zeri: 673
Uni: 564
Due: 1280


In [12]:
yt_int = y_test.astype(int)
# Conto gli zeri, gli uni e i due nell'array
count = np.bincount(yt_int.flatten())

# Stampo i risultati
print(f"Zeri: {count[0]}")
print(f"Uni: {count[1]}")
print(f"Due: {count[2]}")

Zeri: 132
Uni: 107
Due: 241


In [13]:
# processing data for one-hot encoding
def preprocess_data(X, Y):
  X_p = preprocess_input(X)
  Y_p = to_categorical(Y, 3)
  return X_p, Y_p

In [14]:
# one hot encoding applied both to train and test sets
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)

In [15]:
# the model
input_t = Input(shape=(256, 256, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)

In [16]:
# set all the layers not-trainable
for layer in res_model.layers[:]:
  layer.trainable = False

In [17]:
# add trainable layers to performe the classification
model = Sequential()
model.add(res_model)
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [18]:
# compile and fit
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(learning_rate=0.05),
              metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=6, verbose=1)
model.summary()

Epoch 1/6
79/79 [==============================] - 24s 129ms/step - loss: 143.6975 - accuracy: 0.6226
Epoch 2/6
79/79 [==============================] - 9s 119ms/step - loss: 55.8168 - accuracy: 0.7716
Epoch 3/6
79/79 [==============================] - 10s 121ms/step - loss: 28.3318 - accuracy: 0.8530
Epoch 4/6
79/79 [==============================] - 10s 121ms/step - loss: 17.2095 - accuracy: 0.8979
Epoch 5/6
79/79 [==============================] - 10s 121ms/step - loss: 9.5348 - accuracy: 0.9209
Epoch 6/6
79/79 [==============================] - 10s 123ms/step - loss: 8.5189 - accuracy: 0.9337
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 average_pooling2d (AverageP  (None, 7, 7, 2048)       0         
 ooling2D)                                    

In [19]:
# test
test_loss, test_acc = model.evaluate(X_test, y_test)

15/15 [==============================] - 3s 119ms/step - loss: 107.5589 - accuracy: 0.7021
